In [1]:
DOWNLOAD_WORDS = False

In [2]:
from archived_nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
if DOWNLOAD_WORDS:
    nltk.download('words')

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already u

In [3]:
#set this variable to true if you are using light mode, or false if you are using dark mode.
JUPYTER = False
RANDOM_STATE = 42

In [4]:
TF_ENABLE_ONEDNN_OPTS = 0
URLS = CHUNKS_URLS
LIMIT = 10
LARGER_DATASET_PATH = "../datasets"
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_sd_data_chunks_no_stop_words/"
BASE_FILE_NAME = "sd_chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 99744
if LIMIT == 10:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT - 2
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
RANDOM_STATE = 42
USE_READY_DATA = False

### 1.1 Data Sourcing and Munging

#### 1.1.1 Utility Functions

In [5]:
if not os.path.exists(LARGER_DATASET_PATH):
    os.mkdir(LARGER_DATASET_PATH)
if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
    os.mkdir(PREPROCESSED_CHUNKS_PATH)
get_chunks(URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

sd_chunk_1 already exists.
sd_chunk_2 already exists.
sd_chunk_3 already exists.
sd_chunk_4 already exists.
sd_chunk_5 already exists.
sd_chunk_6 already exists.
sd_chunk_7 already exists.
sd_chunk_8 already exists.
sd_chunk_9 already exists.
sd_chunk_10 already exists.


In [6]:
#get all names of downloaded files
all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [7]:
#read all chunks into a list
list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [8]:
#concatenate all chunks into a singular df
df = group_up_chunks(list_dfs)

In [9]:
#check how much of the data was actually downloaded
percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
percent_loaded

100.0

In [10]:
df.columns

Index(['text', 'labels'], dtype='object')

In [11]:
df.reset_index(inplace = True)
df.drop(['index'],axis = 1,inplace = True)

In [12]:
df.isnull().sum()

text      9488
labels       0
dtype: int64

In [13]:
df.dropna(inplace = True)

In [15]:
df['text'].replace('', np.nan, inplace=True)
df.dropna(inplace = True)

In [ ]:
# Extract unique words from the text data
if USE_READY_DATA:
    if os.path.exists("../archived_files/pickle_files/unique_words.pickle"):
        unique_words = pickle.load(open("../archived_files/pickle_files/unique_words.pickle",'rb'))
    else:
        unique_words = set(' '.join(df['text']).split())
else:
    unique_words = set(' '.join(df['text']).split())

In [ ]:
# Count the number of occurrences of each unique word and create a bar chart of the top 20 most frequent unique words
unique_word_count = df['text'].str.split(expand=True).stack().value_counts()
top_unique_words = unique_word_count.loc[unique_word_count.index.isin(unique_words)].head(20)
plt.figure(figsize=(10, 5))

colors = ['b', 'g'] 
shades = [0.37, 0.4, 0.6, 0.8]
palette = [sns.set_hls_values(color, l=s) for color in colors for s in shades]

sns.barplot(x=top_unique_words.values, y=top_unique_words.index,palette=palette)
plt.title('Top 20 Most Frequent Unique Words')
plt.xlabel('Word Count')
plt.ylabel('Word')

In [ ]:
# Create a countplot of the star ratings using Seaborn
visualize_ratings_bar(df['labels'])

In [ ]:
visualize_ratings_pie(df['labels'])

In [ ]:
word_lengths = df['text'].str.split().apply(lambda x: [len(word) for word in x])

In [ ]:
# Creating a Histogram of word lengths
word_lengths = df['text'].str.split().apply(lambda x: [len(word) for word in x])
word_lengths = [length for sublist in word_lengths for length in sublist]
plt.figure(figsize=(10, 6))
plt.hist(word_lengths, bins=10, range=[1, 10], lw=0.2, edgecolor='black')
plt.title('Histogram of Word Lengths')
plt.xlabel('Word Length')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Creating a Histogram of review lengths
review_lengths = [len(text.split()) for text in df['text']]
plt.figure(figsize=(10, 6))
plt.hist(review_lengths, bins=15, range=[50, 350], edgecolor='black')
plt.title('Histogram of Text Lengths by words')
plt.xlabel('Review Length')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()